## ANÁLISE DO CLUSTER 0 DA UP GLÓRIA

In [46]:
# Bibliotecas
import numpy as np
import pandas as pd

In [47]:
# Upload do arquivo para análise
gloria_0_caminho = r"C:\Users\Administrador\Downloads\TCC-20240110T120732Z-001\Dados_UP Gloria\Cluster 0\Cluster0_GLORIA_ALTERACOES.csv"
gloria_0 = pd.read_csv(gloria_0_caminho, sep = ";", encoding= "utf8" )
gloria_0['Alteracao_IT'] = gloria_0['Alteracao_IT'].astype(int)
gloria_0['Alteracao_QH'] = gloria_0['Alteracao_QH'].astype(int)

# Estatísticas descritivas

In [48]:
estatisticas_descritivas = ['Alteracao_QH', 'Alteracao_IT', 'Houve aumento passageiro', 'Tarifa_Aumento', 'Tarifa_reducao']
for col in estatisticas_descritivas:
    gloria_0[col] = gloria_0[col].astype(int)

estatisticas = {}
for col in gloria_0.columns:
   estatisticas[col] = {
        'Total Observações': gloria_0[col].count(),
        '0 - Não': (gloria_0[col] == 0).sum(),
        '1 - Sim': (gloria_0[col] == 1).sum()
    }

estatisticas = pd.DataFrame(estatisticas)
estatisticas


,ID,Bairro,id_hex,Mes,Alteracao_QH,Alteracao_IT,Pass_Transportado,Houve aumento passageiro,Tarifa_Aumento,Tarifa_reducao
Total Observações,1998,1998,1998,92,1998,1998,1998,1998,1998,1998
0 - Não,0,0,0,0,1470,1945,921,1510,1804,1844
1 - Sim,0,0,0,0,528,53,107,488,194,154


# Teste V de Cramer
Objetivo: Testar asssociação entre as variáveis

In [49]:
# Pacotes
from scipy.stats import chi2_contingency

In [50]:
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

In [51]:
columns_to_test = ['Alteracao_QH', 'Alteracao_IT', 'Houve aumento passageiro', 'Tarifa_Aumento', 'Tarifa_reducao']
# Preparando um DataFrame para armazenar os resultados do V de Cramer
cramers_results_selected = pd.DataFrame(index=columns_to_test, columns=columns_to_test)

# Calculando o V de Cramer para cada par de variáveis selecionadas
for col1 in columns_to_test:
    for col2 in columns_to_test:
        cramers_results_selected.loc[col1, col2] = cramers_v(gloria_0[col1], gloria_0[col2])

cramers_results_selected

,Alteracao_QH,Alteracao_IT,Houve aumento passageiro,Tarifa_Aumento,Tarifa_reducao
Alteracao_QH,0.998712,0.055774,0.088273,0.193376,0.326173
Alteracao_IT,0.055774,0.990304,0.0,0.04346,0.035392
Houve aumento passageiro,0.088273,0.0,0.998644,0.016834,0.124199
Tarifa_Aumento,0.193376,0.04346,0.016834,0.997144,0.088846
Tarifa_reducao,0.326173,0.035392,0.124199,0.088846,0.99648


Análise: Associação fraca a moderada entre alterações no Quadro de Horário e redução da tarifa. Não há associação entre o aumento de passageiros e alterações realizadas pela SUMOB ou BHTRANS

# ANÁLISE BINOMIAL NEGATIVA

In [52]:
gloria0_cam = r"C:\Users\Administrador\Downloads\TCC-20240110T120732Z-001\Dados_UP Gloria\Cluster 0\CLUSTER0.xlsx"
gloria0 = pd.read_excel(gloria0_cam)
gloria0

,Hexagono,Bairro,UP,Mes,Aumentou,Quantidade_passageiros(Y),Numero_linhas,Frequencia_HEX,Populacao,Renda_DOMICILIAR,Quantidade_escolas,Cluster
0,89a88cd8537ffff,Glória,Glória,"agosto, 2022",0,61,3,100,1245,668.0,1,0
1,89a88cd8583ffff,Glória,Glória,"agosto, 2022",0,76,3,306,1703,627.0,1,0
2,89a88cd8593ffff,Álvaro Camargos,Glória,"agosto, 2022",0,17,1,52,2192,631.6,0,0
3,89a88cd85afffff,Glória,Glória,"agosto, 2022",0,8,3,94,1420,753.5,0,0
4,89a88cd85b3ffff,Novo Glória,Glória,"agosto, 2022",0,219,3,308,2021,725.7,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
193,89a88cdae67ffff,Coqueiros,Glória,"agosto, 2023",1,619,1,76,1620,595.3,0,0
194,89a88cdae6bffff,Glória,Glória,"agosto, 2023",1,199,3,129,1245,707.7,1,0
195,89a88cdae73ffff,Pindorama,Glória,"agosto, 2023",1,157,2,81,2103,667.8,1,0
196,89a88cdae8bffff,Pindorama,Glória,"agosto, 2023",1,48,2,52,1475,415.4,0,0


In [53]:
#pacotes
import statsmodels.api as sm
from statsmodels.formula.api import negativebinomial

In [55]:
modelo_nb = negativebinomial('Q("Quantidade_passageiros(Y)") ~ Frequencia_HEX + Numero_linhas + Populacao + Renda_DOMICILIAR', data=gloria0)
resultado_nb = modelo_nb.fit()

# Visualizando os resultados
print(resultado_nb.summary())

Optimization terminated successfully.
         Current function value: 5.958400
         Iterations: 22
         Function evaluations: 35
         Gradient evaluations: 27
                           NegativeBinomial Regression Results                            
Dep. Variable:     Q("Quantidade_passageiros(Y)")   No. Observations:                  198
Model:                           NegativeBinomial   Df Residuals:                      193
Method:                                       MLE   Df Model:                            4
Date:                            Sat, 20 Jan 2024   Pseudo R-squ.:                 0.02305
Time:                                    13:14:38   Log-Likelihood:                -1179.8
converged:                                   True   LL-Null:                       -1207.6
Covariance Type:                        nonrobust   LLR p-value:                 2.345e-11
                       coef    std err          z      P>|z|      [0.025      0.975]
---------------

c:\Users\Administrador\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\discrete\discrete_model.py:3377: RuntimeWarning: divide by zero encountered in log
  llf = coeff + size*np.log(prob) + endog*np.log(1-prob)
c:\Users\Administrador\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\discrete\discrete_model.py:3377: RuntimeWarning: divide by zero encountered in log
  llf = coeff + size*np.log(prob) + endog*np.log(1-prob)


Concluí-se que a  frequência tem relação com a quantidade de passageiros, a renda indica pouca influencia. 